In [43]:

from keras import layers
from keras import models
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [44]:
# Каталог с данными для обучения
train_dir = 'cadmnist/train'
# Каталог с данными для проверки
val_dir = 'cadmnist/val'
# Каталог с данными для тестирования
test_dir = 'cadmnist/test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 30
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 188
# Количество изображений для проверки
nb_validation_samples = 89
# Количество изображений для тестирования
nb_test_samples = 86
# количество классов
num_classes=5

In [45]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# model.add(layers.Conv2D(32, (3, 3), input_shape=input_shape))
# model.add(layers.Activation('relu'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(layers.Conv2D(32, (3, 3)))
# model.add(layers.Activation('relu'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(layers.Conv2D(64, (3, 3)))
# model.add(layers.Activation('relu'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(layers.Flatten())
# model.add(layers.Dense(64))
# model.add(layers.Activation('relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(1))
# model.add(layers.Activation('sigmoid'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 74, 74, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_25 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_26 (Conv2D)          (None, 34, 34, 64)        36928     
                                                                 
 flatten_8 (Flatten)         (None, 73984)            

In [46]:
datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 188 images belonging to 5 classes.
Found 89 images belonging to 5 classes.
Found 86 images belonging to 5 classes.


In [47]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [48]:
model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30
23/23 [==============================] - 6s 243ms/step - loss: 39.5599 - accuracy: 0.4667 - val_loss: 1.2204 - val_accuracy: 0.6023
Epoch 2/30
23/23 [==============================] - 5s 203ms/step - loss: 0.7538 - accuracy: 0.7778 - val_loss: 1.0208 - val_accuracy: 0.7500
Epoch 3/30
23/23 [==============================] - 5s 225ms/step - loss: 0.2103 - accuracy: 0.9222 - val_loss: 1.4421 - val_accuracy: 0.7500
Epoch 4/30
23/23 [==============================] - 5s 213ms/step - loss: 0.1021 - accuracy: 0.9667 - val_loss: 1.7018 - val_accuracy: 0.7045
Epoch 5/30
23/23 [==============================] - 5s 211ms/step - loss: 0.0772 - accuracy: 0.9667 - val_loss: 2.2264 - val_accuracy: 0.7386
Epoch 6/30
23/23 [==============================] - 5s 220ms/step - loss: 0.0525 - accuracy: 0.9783 - val_loss: 2.4353 - val_accuracy: 0.7955
Epoch 7/30
23/23 [==============================] - 5s 223ms/step - loss: 0.1492 - accuracy: 0.9722 - val_loss: 2.4649 - val_accuracy: 0.7727
Epoch

In [49]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

C:\Users\Shrek\AppData\Local\Temp\ipykernel_14148\1626445616.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


Точность на тестовых данных: 86.25%


In [50]:
model.save('cadmnist.h5')